# TensorFlow 2 quickstart for geospatial cmoputer vision

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt geospatial dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

In [ ]:
# install required libraries
!pip install -q rasterio
!pip install -q geopandas
!pip install -q radiant_mlhub # for dataset access, see: https://mlhub.earth/

In [114]:
import os, glob, functools, fnmatch, io, shutil, tarfile, json
from zipfile import ZipFile
from itertools import product
from pathlib import Path
import urllib.request
from radiant_mlhub import Dataset
from radiant_mlhub import Dataset, client, get_session, Collection
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.12.0


## Load a dataset

Load and prepare the [Drone Imagery Classification Training Dataset for Crop Types in Rwanda dataset from Radiant Earth](https://mlhub.earth/data/rti_rwanda_crop_type). You'll need a Radiant Earth MLHub API key for this.

In [ ]:
# configure Radiant Earth MLHub access
!mlhub configure

In [5]:
ds = Dataset.fetch('rti_rwanda_crop_type')
for c in ds.collections:
    print(c.id)


rti_rwanda_crop_type_labels
rti_rwanda_crop_type_source
rti_rwanda_crop_type_raw


In [6]:
# Mount google drive.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
# set your root directory and tiled data folders
if 'google.colab' in str(get_ipython()):
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    root_dir = '/content/gdrive/My Drive/tf-eo-devseed-2/'
    workshop_dir = '/content/gdrive/My Drive/tf-eo-devseed-workshop-2'
    dirs = [root_dir, workshop_dir]
    for d in dirs:
        if not os.path.exists(d):
            os.makedirs(d)
    print('Running on Colab')
else:
    root_dir = os.path.abspath("./data/tf-eo-devseed-2")
    workshop_dir = os.path.abspath('./tf-eo-devseed-workshop-2')
    print(f'Not running on Colab, data needs to be downloaded locally at {os.path.abspath(root_dir)}')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Running on Colab


In [ ]:
%cd $root_dir

In [19]:
collections = [
    'rti_rwanda_crop_type_labels',
    'rti_rwanda_crop_type_source'
]

def download(collection_id):
    print(f'Downloading {collection_id}...')
    collection = Collection.fetch(collection_id)
    path = collection.download('.')
    tar = tarfile.open(path, "r:gz")
    tar.extractall()
    tar.close()
    os.remove(path)

def resolve_path(base, path):
    return Path(os.path.join(base, path)).resolve()

def load_df(collection_id):
    collection = json.load(open(f'{collection_id}/collection.json', 'r'))
    rows = []
    item_links = []
    for link in collection['links']:
        if link['rel'] != 'item':
            continue
        item_links.append(link['href'])
    for item_link in item_links:
        item_path = f'{collection_id}/{item_link}'
        current_path = os.path.dirname(item_path)
        item = json.load(open(item_path, 'r'))
        tile_id = item['id'].split('_')[-1]
        for asset_key, asset in item['assets'].items():
            rows.append([
                tile_id,
                None,
                None,
                asset_key,
                str(resolve_path(current_path, asset['href']))
            ])

        for link in item['links']:
            if link['rel'] != 'source':
                continue
            link_path = resolve_path(current_path, link['href'])
            source_path = os.path.dirname(link_path)
            try:
                source_item = json.load(open(link_path, 'r'))
            except FileNotFoundError:
                continue
            datetime = source_item['properties']['datetime']
            satellite_platform = source_item['collection'].split('_')[-1]
            for asset_key, asset in source_item['assets'].items():
                rows.append([
                    tile_id,
                    datetime,
                    satellite_platform,
                    asset_key,
                    str(resolve_path(source_path, asset['href']))
                ])
    return pd.DataFrame(rows, columns=['tile_id', 'datetime', 'satellite_platform', 'asset', 'file_path'])

for c in collections:
    download(c)

train_df = load_df('rti_rwanda_crop_type_labels')
#test_df = load_df('rti_rwanda_crop_type_labels')

  0%|          | 0/0.8 [00:00<?, ?M/s]

  0%|          | 0/104.5 [00:00<?, ?M/s]

In [158]:
# Read the classes
pd.set_option('display.max_colwidth', None)

data = {'class_names':  ['other', 'banana', 'maize', 'legumes', 'forest', 'structure'],
        'class_ids': [0, 1, 2, 3, 4, 5]
        }

classes = pd.DataFrame(data)
print(classes)

classes.to_csv('rti_rwanda_crop_type_classes.csv')

  class_names  class_ids
0       other          0
1      banana          1
2       maize          2
3     legumes          3
4      forest          4
5   structure          5


In [42]:
train_df_labels = train_df.loc[train_df['asset'] == 'labels']

In [43]:
data_train, data_test = train_test_split(train_df_labels, test_size=0.3, random_state=1)

In [44]:
len(data_train), len(data_test)

(1824, 782)

In [180]:
train_image_arrays = []
train_label_integers = []
for i, r in data_train.iterrows():
  label_path = r.file_path
  with open(label_path) as f:
    label_obj = json.load(f)
  label_str = label_obj['label']
  label_int = classes.loc[classes['class_names'] == label_str, 'class_ids'].squeeze()
  train_label_integers.append(label_int)
  image_path = label_path.replace('labels', 'source')
  image_path = image_path.replace('source.json', '')
  image_obj = np.array(Image.open(f"{image_path}/image.png"))
  train_image_arrays.append(image_obj)



In [182]:
test_image_arrays = []
test_label_integers = []
for i, r in data_test.iterrows():
  label_path = r.file_path
  with open(label_path) as f:
    label_obj = json.load(f)
  label_str = label_obj['label']
  label_int = classes.loc[classes['class_names'] == label_str, 'class_ids'].squeeze()
  test_label_integers.append(label_int)
  image_path = label_path.replace('labels', 'source')
  image_path = image_path.replace('source.json', '')
  image_obj = np.array(Image.open(f"{image_path}/image.png"))
  test_image_arrays.append(image_obj)

In [183]:
set(train_label_integers), set(test_label_integers)

({0, 1, 2, 3, 4, 5}, {0, 1, 2, 3, 4, 5})

In [184]:
x_train, y_train = np.array(train_image_arrays), np.array(train_label_integers)
x_test, y_test = np.array(test_image_arrays), np.array(test_label_integers)

In [ ]:
x_test, y_test

## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [186]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(200, 200, 3)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(6)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [187]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 82.98346 , -67.60655 ,  -9.406067, 157.87369 , -69.06694 ,
         80.105225]], dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [188]:
tf.nn.softmax(predictions).numpy()

array([[2.9894212e-33, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        0.0000000e+00, 1.6810692e-34]], dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [189]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/6 for each class), so the initial loss should be close to `-tf.math.log(1/6)`.

In [190]:
loss_fn(y_train[:1], predictions).numpy()

0.0

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [191]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [192]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
57/57 [==============================] - 21s 334ms/step - loss: 5801.9668 - accuracy: 0.2484
Epoch 2/5
57/57 [==============================] - 18s 310ms/step - loss: 1.7813 - accuracy: 0.2873
Epoch 3/5
57/57 [==============================] - 22s 385ms/step - loss: 1.7697 - accuracy: 0.2873
Epoch 4/5
57/57 [==============================] - 18s 310ms/step - loss: 1.7583 - accuracy: 0.2873
Epoch 5/5
57/57 [==============================] - 17s 307ms/step - loss: 1.7475 - accuracy: 0.2873


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [193]:
model.evaluate(x_test,  y_test, verbose=2)

25/25 - 1s - loss: 1.7301 - accuracy: 0.3248 - 1s/epoch - 41ms/step


[1.730060338973999, 0.32480818033218384]

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [194]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [195]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[0.16327359, 0.16869745, 0.19480447, 0.14407335, 0.18652132,
        0.14262979],
       [0.16327359, 0.16869745, 0.19480447, 0.14407335, 0.18652132,
        0.14262979],
       [0.16327359, 0.16869745, 0.19480447, 0.14407335, 0.18652132,
        0.14262979],
       [0.16327359, 0.16869745, 0.19480447, 0.14407335, 0.18652132,
        0.14262979],
       [0.16327359, 0.16869745, 0.19480447, 0.14407335, 0.1865213 ,
        0.14262979]], dtype=float32)>